In [ ]:
import subprocess
import sys
import os
import time
import json
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from pathlib import Path

# Analyze games in python

## Helper

In [ ]:
REPO_ROOT = Path.cwd().parent
BIN_NAME = "50missions"
PACKAGE = "jeu_50missions"
BIN_PATH = REPO_ROOT / "target" / "release" / BIN_NAME
DATA_PATH = REPO_ROOT / "log"

In [ ]:
def cargo_build():
    try:
        subprocess.run(["cargo", "build", "--release"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error during cargo build: {e}")
        sys.exit(1)

def cargo_run(args: str|list):
    if isinstance(args, str):
        args = args.split(" ")
    try:
        result = subprocess.run(["cargo", "run", "--release", "--bin", BIN_NAME, "--", *args], check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error during cargo run: {e}")
        print(f"Stdout:")
        print(e.stdout)
        print(f"\nStderr:")
        print(e.stderr)
        sys.exit(1)

def maybe(flag, value=None):
    if value is False or value is None:
        return []
    if value is True:
        return [flag]
    return [flag, str(value)]

def run_simulation(
        verbose=False,
        use_random_strategy=False,
        batch_size=10,
        seed=None,
        output=None,
        parallel_depth=3,
        sequential_depth=4,
    ):
    if output is None:
        ts = time.strftime("%Y%m%d_%H%M%S")
        output = DATA_PATH / f'simulation_single_run_{ts}.json'

    assert not output.exists(), f"Output file {output} already exists, use another path or delete the existing file"
    cmd = []
    cmd += maybe("--verbose", verbose)
    cmd += maybe("--random", use_random_strategy)
    cmd += maybe("--batch-size", batch_size)
    cmd += maybe("--seed", seed)
    cmd += maybe("--par-depth", parallel_depth)
    cmd += maybe("--seq-depth", sequential_depth)

    cmd += ["--output", output]
    cargo_run(cmd)
    assert output.exists(), f"Expected output file {output} does not exist"

    return output

## Run and analysis

In [ ]:
# Generate a simulation result
file = run_simulation(
  batch_size=1 << 10,
  use_random_strategy=False,
  verbose=False,
  seed=42,
  parallel_depth=0,
  sequential_depth=4,
)

In [ ]:
# make a cdf and histogram of the completed missions
with open(file) as f:
    data = json.load(f)
    print(f"Loaded {len(data)} simulation results")
    completed_missions = pd.DataFrame([res['final_state']['completed_missions'] for res in data], columns=['completed_missions'])

fig = px.histogram(completed_missions, x='completed_missions', nbins=50, title='Distribution of Completed Missions')
fig.update_layout(xaxis_title='Number of Completed Missions', yaxis_title='Count')
fig.show()

fig = px.ecdf(completed_missions, x='completed_missions', title='CDF of Completed Missions')
fig.update_layout(xaxis_title='Number of Completed Missions', yaxis_title='Cumulative Probability')
fig.show()

In [ ]:
# Histogram of run-ending missions
end_mission_counter = {}
with open(file) as f:
    data = json.load(f)
    for res in data:
        final_state = res['final_state']
        for mission_name in final_state["table_missions"]:
            end_mission_counter[mission_name] = end_mission_counter.get(mission_name, 0) + 1

df = pd.DataFrame(list(end_mission_counter.items()), columns=['mission_name', 'count'])
df = df.sort_values(by='count', ascending=False)
fig = px.bar(df, x='mission_name', y='count', title='Frequency of Ending Missions')
fig.update_layout(xaxis_title='Mission Name', yaxis_title='Count')
fig.show()